In [2]:
import numpy as np
import matplotlib.pyplot as plt
from jax import random
import jax.numpy as jnp
from neural_tangents import stax

# Generate dataset
x_train = np.linspace(-np.pi, np.pi, 400).reshape(-1, 1)
y_train = np.sin(x_train)

x_test = np.linspace(-1.5*np.pi, 1.5*np.pi, 1000).reshape(-1, 1)



In [9]:
def FullyConnectedNetwork(depth=7):
    layers = []
    for _ in range(depth):
        layers.append(stax.Dense(512))  # Width is 512 (can be adjusted)
        layers.append(stax.Relu())
    layers.append(stax.Dense(1))
    return stax.serial(*layers)

init_fn, apply_fn, kernel_fn = FullyConnectedNetwork()


In [11]:
from neural_tangents import predict

ntk_train_train = kernel_fn(x_train, x_train, 'ntk')
ntk_test_train = kernel_fn(x_test, x_train, 'ntk')
ntk_test_test = kernel_fn(x_test, x_test, 'ntk')

# Train using the NTK
predictor = predict.gradient_descent_mse(ntk_train_train, y_train)
y_test_ntk = predictor(t=1.0, kernel=ntk_test_train)






TypeError: predict_fn() got an unexpected keyword argument 'kernel'